In [11]:
# Importing useful packages
import numpy as np
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from new_var import calc,y1function,y2function,C3function,C6function
# from outliers import outlier
# from Standardising import standard
#from Recoding_SIC_Codes import Industry_Division
%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support


# Read in Data file and define NaN values
housetrain = pd.read_csv("train.csv",header=0,na_values='None')
housetrain.MSSubClass = housetrain.MSSubClass.astype(str)

housetest = pd.read_csv("train.csv",header=0,na_values='None')
housetest.MSSubClass = housetrain.MSSubClass.astype(str)

In [12]:
# training set frequncy counts
# Graphing missing data
group = housetrain.columns.to_series().groupby(housetrain.dtypes).groups # grouping columns by type
groups={k.name: v for k, v in group.items()} #  creating as dictionary

# Taking only the object type col names
objects=housetrain[groups['object'].values]
#print(objects.head(5))
# Printing freqiency counts
for i in objects.columns:
        #print('{} \n' .format(objects[i]))
        print(objects[i].value_counts())
        print('\n')
        
        


20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64


RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64


Pave    1454
Grvl       6
Name: Street, dtype: int64


Grvl    50
Pave    41
Name: Alley, dtype: int64


Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64


Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64


AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64


Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64


Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64


NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel

In [13]:
# testing set frequency counts

group = housetest.columns.to_series().groupby(housetest.dtypes).groups # grouping columns by type
groups={k.name: v for k, v in group.items()} #  creating as dictionary

# Taking only the object type col names
objects=housetest[groups['object'].values]
#print(objects.head(5))
# Printing freqiency counts
for i in objects.columns:
        #print('{} \n' .format(objects[i]))
        print(objects[i].value_counts())
        print('\n')

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64


RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64


Pave    1454
Grvl       6
Name: Street, dtype: int64


Grvl    50
Pave    41
Name: Alley, dtype: int64


Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64


Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64


AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64


Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64


Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64


NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel

In [14]:
# Recoding and re looking at freq counts training set
# reg or irreg
housetrain['LotShape']=housetrain['LotShape'].replace(['IR1','IR2','IR3'],'IRReg')
#print(housetrain['LotShape'].value_counts())

# flat or not flat
housetrain['LandContour']=housetrain['LandContour'].replace(['Bnk','HLS','Low'],'NotFlat')
#print(housetrain['LandContour'].value_counts())

# combined frontage
housetrain['LotConfig']=housetrain['LotConfig'].replace(['FR2','FR3'],'Frontage')
#print(housetrain['LotConfig'].value_counts())

# combined rail and pos
housetrain['Condition1']=housetrain['Condition1'].replace(['RRNn','RRAn','RRNe','RRAe'],'Rail')
housetrain['Condition1']=housetrain['Condition1'].replace(['PosN','PosA'],'Pos')
#print(housetrain['Condition1'].value_counts())

# combined rail and pos
housetrain['Condition2']=housetrain['Condition2'].replace(['RRNn','RRAn','RRNe','RRAe'],'Rail')
housetrain['Condition2']=housetrain['Condition2'].replace(['PosN','PosA'],'Pos')
#print(housetrain['Condition2'].value_counts())

# Recoding to have less options and grouping similar
housetrain['ExterQual']=housetrain['ExterQual'].replace(['Ex','Gd'],'Above Average')
housetrain['ExterQual']=housetrain['ExterQual'].replace(['Fa','Po'],'Below Average')
#print(housetrain['ExterQual'].value_counts())

# Recoding to have less options and grouping similar
housetrain['ExterCond']=housetrain['ExterCond'].replace(['Ex','Gd'],'Above Average')
housetrain['ExterCond']=housetrain['ExterCond'].replace(['Fa','Po'],'Below Average')
#print(housetrain['ExterCond'].value_counts())

housetrain['HouseStyle']=housetrain['HouseStyle'].replace(['1Story','1.5Unf','1.5Fin'],'1to2Story')
housetrain['HouseStyle']=housetrain['HouseStyle'].replace(['2Story','2.5Unf','2.5Fin'],'2+Story')
#print(housetrain['HouseStyle'].value_counts())

housetrain['RoofStyle']=housetrain['RoofStyle'].replace(['Flat','Gambrel','Mansard','Shed'],'Other')
#print(housetrain['RoofStyle'].value_counts())

housetrain['RoofMatl']=housetrain['RoofMatl'].replace(['ClyTile','Membran','Metal','Roll','Tar&Grv','WdShake','WdShngl'],'Other')
#print(housetrain['RoofMatl'].value_counts())

# Recoding to have less options and grouping similar
housetrain['SaleType']=housetrain['SaleType'].replace(['WD','CWD','VWD'],'Warrenty Deed')
housetrain['SaleType']=housetrain['SaleType'].replace(['Con','ConLw','ConLI','ConLD'],'Contract')
#print(housetrain['SaleType'].value_counts())

# Recoding to have less options and grouping similar
housetrain['GarageCond']=housetrain['GarageCond'].replace(['Ex','Gd'],'Above Average')
housetrain['GarageCond']=housetrain['GarageCond'].replace(['Fa','Po'],'Below Average')
#print(housetrain['GarageCond'].value_counts())

# Recoding to have less options and grouping similar
housetrain['GarageQual']=housetrain['GarageQual'].replace(['Ex','Gd'],'Above Average')
housetrain['GarageQual']=housetrain['GarageQual'].replace(['Fa','Po'],'Below Average')
#print(housetrain['GarageQual'].value_counts())

# Recoding to have less options and grouping similar
housetrain['Functional']=housetrain['Functional'].replace(['Min1','Min2'],'Min')
housetrain['Functional']=housetrain['Functional'].replace(['Maj1','Maj2','Sev','Sal'],'Maj')
#print(housetrain['Functional'].value_counts())

# Recoding to have less options and grouping similar
housetrain['KitchenQual']=housetrain['KitchenQual'].replace(['Ex','Gd'],'Above Average')
housetrain['KitchenQual']=housetrain['KitchenQual'].replace(['Fa','Po'],'Below Average')
#print(housetrain['KitchenQual'].value_counts())

# Recoding to have less options and grouping similar
housetrain['HeatingQC']=housetrain['HeatingQC'].replace(['Ex','Gd'],'Above Average')
housetrain['HeatingQC']=housetrain['HeatingQC'].replace(['Fa','Po'],'Below Average')
#print(housetrain['HeatingQC'].value_counts())

# Merging Gas
housetrain['Heating']=housetrain['Heating'].replace(['GasA','GasW'],'Gas')
#print(housetrain['Heating'].value_counts())

# Recoding to have less options and grouping similar
housetrain['BsmtFinType2']=housetrain['BsmtFinType2'].replace(['ALQ','Rec'],'Average')
housetrain['BsmtFinType2']=housetrain['BsmtFinType2'].replace(['BLQ','LwQ'],'Below Average')
#print(housetrain['BsmtFinType2'].value_counts())

# Recoding to have less options and grouping similar
housetrain['BsmtFinType1']=housetrain['BsmtFinType1'].replace(['ALQ','Rec'],'Average')
housetrain['BsmtFinType1']=housetrain['BsmtFinType1'].replace(['BLQ','LwQ'],'Below Average')
#print(housetrain['BsmtFinType1'].value_counts())

# Recoding to have less options and grouping similar
housetrain['BsmtCond']=housetrain['BsmtCond'].replace(['Ex','Gd'],'Above Average')
housetrain['BsmtCond']=housetrain['BsmtCond'].replace(['Fa','Po'],'Below Average')
#print(housetrain['BsmtCond'].value_counts())

# Recoding to have less options and grouping similar
housetrain['BsmtQual']=housetrain['BsmtQual'].replace(['Ex','Gd'],'Above Average')
housetrain['BsmtQual']=housetrain['BsmtQual'].replace(['Fa','Po'],'Below Average')
#print(housetrain['BsmtQual'].value_counts())

# Foundation: One of the more standard options or other
housetrain['Foundation']=housetrain['Foundation'].replace(['BrkTil','Slab','Stone','Wood'],'Other')
# Encode some categorical features as ordered numbers when there is information in the order
housetrain = housetrain.replace({"BsmtCond" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                        "Fence":{"GdPrv":2,"GdWo":2,"MnPrv":1,"MnWw":1,"No":0},
                        "LotShape":{"IRReg":0,"Reg":1},
                       "BsmtQual" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "ExterCond" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "ExterQual" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "BsmtFinType1":{"No":0,"Unf":1,"Below Average":1,"Average":2,"GLQ":3},
                        "BsmtFinType2":{"No":0,"Unf":1,"Below Average":1,"Average":2,"GLQ":3},
                       "Functional" : {"Maj" : 1, "Mod" : 2, "Min" : 3, "Typ" : 4},
                       "GarageCond" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "GarageQual" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "HeatingQC" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "KitchenQual" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3}}
                     )

# Create new features
# 1* Simplifications of existing features
housetrain["OverallQual"] = housetrain.OverallQual.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })
housetrain["OverallCond"] = housetrain.OverallCond.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })



group = housetrain.columns.to_series().groupby(housetrain.dtypes).groups # grouping columns by type
groups={k.name: v for k, v in group.items()} #  creating as dictionary

# Taking only the object type col names
objects=housetrain[groups['object'].values]
for i in objects.columns:
        #print('{} \n' .format(objects[i]))
        print(objects[i].value_counts())
        print('\n')

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64


RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64


Pave    1454
Grvl       6
Name: Street, dtype: int64


Grvl    50
Pave    41
Name: Alley, dtype: int64


Lvl        1311
NotFlat     149
Name: LandContour, dtype: int64


AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64


Inside      1052
Corner       263
CulDSac       94
Frontage      51
Name: LotConfig, dtype: int64


NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
StoneBr     25
SWISU       25
MeadowV     17
Blmngtn     17
BrDale      16
Veenker     11
NPkVill      

In [15]:
# same as above but for testing

# reg or irreg
housetest['LotShape']=housetest['LotShape'].replace(['IR1','IR2','IR3'],'IRReg')
#print(housetest['LotShape'].value_counts())

# flat or not flat
housetest['LandContour']=housetest['LandContour'].replace(['Bnk','HLS','Low'],'NotFlat')
#print(housetest['LandContour'].value_counts())

# combined frontage
housetest['LotConfig']=housetest['LotConfig'].replace(['FR2','FR3'],'Frontage')
#print(housetest['LotConfig'].value_counts())

# combined rail and pos
housetest['Condition1']=housetest['Condition1'].replace(['RRNn','RRAn','RRNe','RRAe'],'Rail')
housetest['Condition1']=housetest['Condition1'].replace(['PosN','PosA'],'Pos')
#print(housetest['Condition1'].value_counts())

# combined rail and pos
housetest['Condition2']=housetest['Condition2'].replace(['RRNn','RRAn','RRNe','RRAe'],'Rail')
housetest['Condition2']=housetest['Condition2'].replace(['PosN','PosA'],'Pos')
#print(housetest['Condition2'].value_counts())

# Recoding to have less options and grouping similar
housetest['ExterQual']=housetest['ExterQual'].replace(['Ex','Gd'],'Above Average')
housetest['ExterQual']=housetest['ExterQual'].replace(['Fa','Po'],'Below Average')
#print(housetest['ExterQual'].value_counts())

# Recoding to have less options and grouping similar
housetest['ExterCond']=housetest['ExterCond'].replace(['Ex','Gd'],'Above Average')
housetest['ExterCond']=housetest['ExterCond'].replace(['Fa','Po'],'Below Average')
#print(housetest['ExterCond'].value_counts())

housetest['HouseStyle']=housetest['HouseStyle'].replace(['1Story','1.5Unf','1.5Fin'],'1to2Story')
housetest['HouseStyle']=housetest['HouseStyle'].replace(['2Story','2.5Unf','2.5Fin'],'2+Story')
#print(housetest['HouseStyle'].value_counts())

housetest['RoofStyle']=housetest['RoofStyle'].replace(['Flat','Gambrel','Mansard','Shed'],'Other')
#print(housetest['RoofStyle'].value_counts())

housetest['RoofMatl']=housetest['RoofMatl'].replace(['ClyTile','Membran','Metal','Roll','Tar&Grv','WdShake','WdShngl'],'Other')
#print(housetest['RoofMatl'].value_counts())

# Recoding to have less options and grouping similar
housetest['SaleType']=housetest['SaleType'].replace(['WD','CWD','VWD'],'Warrenty Deed')
housetest['SaleType']=housetest['SaleType'].replace(['Con','ConLw','ConLI','ConLD'],'Contract')
#print(housetest['SaleType'].value_counts())

# Recoding to have less options and grouping similar
housetest['GarageCond']=housetest['GarageCond'].replace(['Ex','Gd'],'Above Average')
housetest['GarageCond']=housetest['GarageCond'].replace(['Fa','Po'],'Below Average')
#print(housetest['GarageCond'].value_counts())

# Recoding to have less options and grouping similar
housetest['GarageQual']=housetest['GarageQual'].replace(['Ex','Gd'],'Above Average')
housetest['GarageQual']=housetest['GarageQual'].replace(['Fa','Po'],'Below Average')
#print(housetest['GarageQual'].value_counts())

# Recoding to have less options and grouping similar
housetest['Functional']=housetest['Functional'].replace(['Min1','Min2'],'Min')
housetest['Functional']=housetest['Functional'].replace(['Maj1','Maj2','Sev','Sal'],'Maj')
#print(housetest['Functional'].value_counts())

# Recoding to have less options and grouping similar
housetest['KitchenQual']=housetest['KitchenQual'].replace(['Ex','Gd'],'Above Average')
housetest['KitchenQual']=housetest['KitchenQual'].replace(['Fa','Po'],'Below Average')
#print(housetest['KitchenQual'].value_counts())

# Recoding to have less options and grouping similar
housetest['HeatingQC']=housetest['HeatingQC'].replace(['Ex','Gd'],'Above Average')
housetest['HeatingQC']=housetest['HeatingQC'].replace(['Fa','Po'],'Below Average')
#print(housetest['HeatingQC'].value_counts())

# Merging Gas
housetest['Heating']=housetest['Heating'].replace(['GasA','GasW'],'Gas')
#print(housetest['Heating'].value_counts())

# Recoding to have less options and grouping similar
housetest['BsmtFinType2']=housetest['BsmtFinType2'].replace(['ALQ','Rec'],'Average')
housetest['BsmtFinType2']=housetest['BsmtFinType2'].replace(['BLQ','LwQ'],'Below Average')
#print(housetest['BsmtFinType2'].value_counts())

# Recoding to have less options and grouping similar
housetest['BsmtFinType1']=housetest['BsmtFinType1'].replace(['ALQ','Rec'],'Average')
housetest['BsmtFinType1']=housetest['BsmtFinType1'].replace(['BLQ','LwQ'],'Below Average')
#print(housetest['BsmtFinType1'].value_counts())

# Recoding to have less options and grouping similar
housetest['BsmtCond']=housetest['BsmtCond'].replace(['Ex','Gd'],'Above Average')
housetest['BsmtCond']=housetest['BsmtCond'].replace(['Fa','Po'],'Below Average')
#print(housetest['BsmtCond'].value_counts())

# Recoding to have less options and grouping similar
housetest['BsmtQual']=housetest['BsmtQual'].replace(['Ex','Gd'],'Above Average')
housetest['BsmtQual']=housetest['BsmtQual'].replace(['Fa','Po'],'Below Average')
#print(housetest['BsmtQual'].value_counts())

# Foundation: One of the more standard options or other
housetest['Foundation']=housetest['Foundation'].replace(['BrkTil','Slab','Stone','Wood'],'Other')
# Encode some categorical features as ordered numbers when there is information in the order
housetest = housetest.replace({"BsmtCond" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                        "Fence":{"GdPrv":2,"GdWo":2,"MnPrv":1,"MnWw":1,"No":0},
                        "LotShape":{"IRReg":0,"Reg":1},
                       "BsmtQual" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "ExterCond" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "ExterQual" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "BsmtFinType1":{"No":0,"Unf":1,"Below Average":1,"Average":2,"GLQ":3},
                        "BsmtFinType2":{"No":0,"Unf":1,"Below Average":1,"Average":2,"GLQ":3},
                       "Functional" : {"Maj" : 1, "Mod" : 2, "Min" : 3, "Typ" : 4},
                       "GarageCond" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "GarageQual" : {"No" : 0, "Below Average" : 1, "TA" : 2, "Above Average":3},
                       "HeatingQC" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "KitchenQual" : {"Below Average" : 1, "TA" : 2, "Above Average":3},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3}}
                     )

# Create new features
# 1* Simplifications of existing features
housetest["OverallQual"] = housetest.OverallQual.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })
housetest["OverallCond"] = housetest.OverallCond.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })



group = housetest.columns.to_series().groupby(housetest.dtypes).groups # grouping columns by type
groups={k.name: v for k, v in group.items()} #  creating as dictionary

# Taking only the object type col names
objects=housetest[groups['object'].values]
for i in objects.columns:
        #print('{} \n' .format(objects[i]))
        print(objects[i].value_counts())
        print('\n')

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64


RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64


Pave    1454
Grvl       6
Name: Street, dtype: int64


Grvl    50
Pave    41
Name: Alley, dtype: int64


Lvl        1311
NotFlat     149
Name: LandContour, dtype: int64


AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64


Inside      1052
Corner       263
CulDSac       94
Frontage      51
Name: LotConfig, dtype: int64


NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
StoneBr     25
SWISU       25
MeadowV     17
Blmngtn     17
BrDale      16
Veenker     11
NPkVill      

0          NormalNormalNormal
1                NormalNormal
2          NormalNormalNormal
3       AbnormlAbnormlAbnorml
4          NormalNormalNormal
5                NormalNormal
6          NormalNormalNormal
7          NormalNormalNormal
8       AbnormlAbnormlAbnorml
9                NormalNormal
10               NormalNormal
11      PartialPartialPartial
12               NormalNormal
13      PartialPartialPartial
14               NormalNormal
15         NormalNormalNormal
16               NormalNormal
17               NormalNormal
18               NormalNormal
19             AbnormlAbnorml
20      PartialPartialPartial
21         NormalNormalNormal
22         NormalNormalNormal
23               NormalNormal
24               NormalNormal
25         NormalNormalNormal
26               NormalNormal
27         NormalNormalNormal
28               NormalNormal
29               NormalNormal
                ...          
1430             NormalNormal
1431             NormalNormal
1432      